<a href="https://colab.research.google.com/github/awaiskhan005/DEEP-LEARNING-AND-FORECASTING/blob/main/Gitty_preoject_btc_predition_goolgle_drive_save.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
# Import Required Libraries
import os
import joblib
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Install the 'ta' library if it's not already installed
!pip install ta

# Install the 'python-binance' library if it's not already installed
!pip install python-binance # This line is added to install the missing library

# Now import from 'ta'
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
from ta.trend import EMAIndicator
from binance.client import Client # This import should now work
from datetime import datetime, timedelta, timezone
import time

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=2d074d0d9babc2361ea014ff908c73611afcba317a9922049a29d1e3dc86244a
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 51.6 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from binance.client import Client
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from google.colab import drive
import os

# Binance API Key Setup (Replace with your own keys)
api_key = "VQnnaxhApWnPhoZCRAzzDEfSgibQolyReWc2zPw5lcO5SZUgzyczH25aWyoM7zaA"
api_secret = "MyCOt66HhaTcPFnrnVLWszUGZhRsitrP7RamgdLuXCVaPKff30T|pTC0FCFphOqZ"
client = Client(api_key, api_secret)

# Mount Google Drive to store files persistently
drive.mount('/content/drive')
base_path = "/content/drive/My Drive/BTC_Predictions/"
os.makedirs(base_path, exist_ok=True)

# Step 1: Data Collection from Binance

def fetch_data(symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_5MINUTE, lookback='60 day ago UTC'):
    klines = client.get_historical_klines(symbol, interval, lookback)
    data = pd.DataFrame(klines, columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'CloseTime', 'QuoteAssetVolume', 'NumberOfTrades', 'TakerBuyBaseAssetVolume', 'TakerBuyQuoteAssetVolume', 'Ignore'])
    data['Close'] = data['Close'].astype(float)
    data['Time'] = pd.to_datetime(data['Time'], unit='ms')
    data.set_index('Time', inplace=True)
    return data[['Close']]

# Step 2: Feature Engineering

def create_features(data):
    data['Return'] = data['Close'].pct_change()
    data['MA7'] = data['Close'].rolling(window=7).mean()
    data['MA21'] = data['Close'].rolling(window=21).mean()
    data['Volatility'] = data['Return'].rolling(window=21).std()
    data.dropna(inplace=True)
    return data

# Step 3: Data Preprocessing

def preprocess_data(data):
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(data)
    return scaled_data, scaler

# Step 4: Create LSTM Sequences

def create_sequences(data, lookback=48):
    X, y = [], []
    for i in range(len(data) - lookback - 1):
        X.append(data[i:(i+lookback)])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

# Step 5: Model Development

def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Step 6: Monte Carlo Simulation

def monte_carlo_simulation(initial_price, sigma, time_length=288, num_simulations=100):
    dt = 5 / (24 * 60)
    price_paths = []
    for _ in range(num_simulations):
        price_changes = np.random.normal(0, sigma * np.sqrt(dt), time_length)
        price_path = initial_price * np.cumprod(1 + price_changes)
        price_paths.append(price_path)
    return np.array(price_paths)

# === STEP 1: Run this NOW to generate predictions ===
# Fetch Data
data = fetch_data()
data = create_features(data)
scaled_data, scaler = preprocess_data(data)
X, y = create_sequences(scaled_data)

# Train LSTM Model
model = build_lstm_model((X.shape[1], X.shape[2]))
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.1)

# Monte Carlo Simulation
sigma = data['Volatility'].mean()
initial_price = data['Close'].iloc[-1]
simulated_paths = monte_carlo_simulation(initial_price, sigma)

# Generate Timestamps for the Next 24 Hours
future_times = pd.date_range(start=pd.Timestamp.now(), periods=288, freq='5T')

# Save Predictions for Future Use to Google Drive
predictions_df = pd.DataFrame(simulated_paths.T, index=future_times)
predictions_df.to_csv(os.path.join(base_path, "predicted_paths.csv"))
print("Predictions saved! Fetch real data after 24 hours for comparison.")

# Plot Results with Plotly
fig = go.Figure()
for i, path in enumerate(simulated_paths):
    fig.add_trace(go.Scatter(x=future_times, y=path, mode='lines', name=f'Trace {i+1}', opacity=0.1))

fig.update_layout(
    title='100 Possible Predicted Bitcoin Price Paths (Monte Carlo)',
    xaxis_title='Date and Time (5-Min Intervals for Next 24 Hours)',
    yaxis_title='Price (USD)',
    xaxis=dict(tickformat='%d-%b %H:%M', showgrid=True),
    template='plotly_dark'
)
fig.show()




BinanceAPIException: APIError(code=0): Service unavailable from a restricted location according to 'b. Eligibility' in https://www.binance.com/en/terms. Please contact customer service if you believe you received this message in error.